In [1]:
import pandas as pd

df_bpi = pd.read_csv("../data/BPI_Challenge_2017_processed.csv")
df_bpi['time'] = pd.to_datetime(df_bpi['time'], errors='coerce')
df_bpi

C:\Users\20203666\AppData\Local\Temp\ipykernel_16688\2990813604.py:3: DtypeWarning: Columns (93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204) have mixed types. Specify dtype option on import or set low_memory=False.
  df_bpi = pd.read_csv("../data/BPI_Challenge_2017_processed.csv")


,Action,org:resource,event,EventOrigin,EventID,lifecycle:transition,time,case:LoanGoal,case:ApplicationType,case,...,event_171,event_172,event_173,event_174,event_175,event_176,event_177,event_178,event_179,event_180
0,Created,User_1,A_Create Application,Application,Application_652823628,complete,2016-01-01 09:51:15.304000+00:00,Existing loan takeover,New credit,Application_652823628,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,statechange,User_1,A_Submitted,Application,ApplState_1582051990,complete,2016-01-01 09:51:15.352000+00:00,Existing loan takeover,New credit,Application_652823628,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Created,User_1,W_Handle leads,Workflow,Workitem_1298499574,schedule,2016-01-01 09:51:15.774000+00:00,Existing loan takeover,New credit,Application_652823628,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Deleted,User_1,W_Handle leads,Workflow,Workitem_1673366067,withdraw,2016-01-01 09:52:36.392000+00:00,Existing loan takeover,New credit,Application_652823628,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Created,User_1,W_Complete application,Workflow,Workitem_1493664571,schedule,2016-01-01 09:52:36.403000+00:00,Existing loan takeover,New credit,Application_652823628,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202261,Released,User_96,W_Call after offers,Workflow,Workitem_146325658,suspend,2017-01-02 19:29:25.989000+00:00,Home improvement,New credit,Application_1350494635,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1202262,Deleted,User_1,W_Call after offers,Workflow,Workitem_1817549786,ate_abort,2017-01-06 06:33:02.212000+00:00,Home improvement,New credit,Application_1350494635,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1202263,Created,User_1,W_Call after offers,Workflow,Workitem_363876066,schedule,2017-01-06 06:33:02.221000+00:00,Home improvement,New credit,Application_1350494635,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1202264,statechange,User_28,A_Cancelled,Application,ApplState_1869071797,complete,2017-01-16 09:51:21.114000+00:00,Home improvement,New credit,Application_1350494635,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
lst_case = list(df_bpi['case'].unique())
train_index = round(len(lst_case)*0.8)

df_train_unfiltered = df_bpi[df_bpi['case'].isin(lst_case[:train_index])]
df_test = df_bpi[df_bpi['case'].isin(lst_case[train_index:])]

In [4]:
def split_data(df_train,df_test):
    max_time_train = min(df_test['time'])
    cases = []
    i=0
    for case in list(df_train['case'].unique()):
        if df_train[df_train['case']==case].iloc[-1,6] > max_time_train:
            cases.append(case)
        i+=1
        print(i)
    df_train3 = df_train[~df_train['case'].isin(cases)]
    df_train3.reset_index(inplace=True, drop=True)
    return df_train3

def split_data(df_train, df_test):
    max_time_train = df_test['time'].min()
    # Get the last event (row) for each case
    last_rows = df_train.groupby('case', as_index=False).tail(1)

    # Identify cases where the last event time is greater than max_time_train
    cases_to_exclude = last_rows.loc[last_rows['time'] > max_time_train, 'case']

    # Exclude these cases and reset index
    df_train3 = df_train[~df_train['case'].isin(cases_to_exclude)].reset_index(drop=True)
    return df_train3

In [5]:
df_train = split_data(df_train_unfiltered, df_test)

In [6]:
len(df_train)/len(df_train_unfiltered)

0.9088209835956421

In [7]:
df_train.tail()

,Action,org:resource,event,EventOrigin,EventID,lifecycle:transition,time,case:LoanGoal,case:ApplicationType,case,...,event_171,event_172,event_173,event_174,event_175,event_176,event_177,event_178,event_179,event_180
873561,statechange,User_133,A_Validating,Application,ApplState_1893772850,complete,2016-10-17 14:19:25.237000+00:00,Unknown,New credit,Application_601436698,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
873562,Released,User_133,W_Validate application,Workflow,Workitem_1967558349,suspend,2016-10-17 14:19:48.641000+00:00,Unknown,New credit,Application_601436698,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
873563,statechange,User_75,O_Accepted,Offer,OfferState_326553445,complete,2016-10-17 15:14:12.345000+00:00,Unknown,New credit,Application_601436698,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
873564,statechange,User_75,A_Pending,Application,ApplState_513993826,complete,2016-10-17 15:14:12.348000+00:00,Unknown,New credit,Application_601436698,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
873565,Deleted,User_75,W_Validate application,Workflow,Workitem_878751104,ate_abort,2016-10-17 15:14:12.351000+00:00,Unknown,New credit,Application_601436698,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df_test[df_test['event']=='A_Cancelled']['org:resource'].describe()

count       2000
unique        70
top       User_1
freq        1522
Name: org:resource, dtype: object

In [19]:
last_event_cases = df_train.groupby('case', as_index=False).tail(1)
probability_A_Cancelled = last_event_cases['A_Cancelled_occured'].sum() / last_event_cases['A_Cancelled_occured'].count()

In [20]:
probability_A_Cancelled

np.float64(0.32462882096069867)

In [1]:
!pip install pyarrow


   ---------------------------------------- 0.0/25.1 MB ? eta -:--:--
   ---- ----------------------------------- 2.9/25.1 MB 15.2 MB/s eta 0:00:02
   ---------- ----------------------------- 6.6/25.1 MB 16.1 MB/s eta 0:00:02
   --------------- ------------------------ 10.0/25.1 MB 15.9 MB/s eta 0:00:01
   -------------------- ------------------- 13.1/25.1 MB 15.8 MB/s eta 0:00:01
   -------------------------- ------------- 16.5/25.1 MB 15.8 MB/s eta 0:00:01
   ------------------------------- -------- 19.7/25.1 MB 15.5 MB/s eta 0:00:01
   ------------------------------------ --- 22.8/25.1 MB 15.3 MB/s eta 0:00:01
   ---------------------------------------  24.9/25.1 MB 15.5 MB/s eta 0:00:01
   ---------------------------------------- 25.1/25.1 MB 14.5 MB/s eta 0:00:00


In [2]:
!pip install seaborn
